In [1]:
%%capture

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")



from caregiver import id_parent_served, parents, sdata, id_parent_completed

In [ ]:

id_parent_served.drop_duplicates('id_patient',inplace=True)
id_parent_completed.drop_duplicates('id_patient',inplace=True)
parents.drop_duplicates('code',inplace=True)
sdata.drop_duplicates('case_id',inplace=True)

In [3]:
id_parent_served.id_patient.count()
id_parent_completed.id_patient.count()
parents.code.count()
sdata.case_id.count()



235

101

365

67100

In [4]:
lc_idparentserved = sdata[sdata.id_patient.isin(id_parent_served.id_patient)]
lc_ips = lc_idparentserved[['id_patient','code']]
lc_idparentserved.code.count()
lc_ips.code.count()

235

235

In [5]:
code_in_parents = lc_ips[lc_ips.code.isin(parents.code)]
code_notIn_parents = lc_ips[~lc_ips.code.isin(parents.code)]


code_in_parents.code.count()
code_notIn_parents.code.count()

102

133

# the completed

In [22]:
id_parent_completed[id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()
id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()



importantMissing = id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)]
importantMissing = sdata[sdata.id_patient.isin(importantMissing.id_patient)]
importantMissing.to_excel('./missing/caregiver_CP_missing.xlsx',index=False,na_rep="")

75

26


# COnclusion


- fosref must add "Code_notIn_parents" for the caregiver
- code_in_parents are Datim whorty

## not in parents

In [6]:
missing_FAP = sdata[sdata.id_patient.isin(code_notIn_parents.id_patient)]
missing_FAP.id_patient.count()

133

# in parents - datim

In [7]:
found_FAP = sdata[sdata.id_patient.isin(code_in_parents.id_patient)]
found_FAP.id_patient.count()

cip = code_in_parents[['code']]
parents_reportable = cip.merge(parents,how='left',on='code')
parents_reportable.code.count()

102

102

In [8]:
parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)

Gender,Female,Male
age_ovc,18+,18+
commune,,
delmas,1,0
desdunes,12,8
dessalines,3,0
grande-saline,6,0
gressier,8,0
kenscoff,24,10
la_chapelle,1,0
montrouis,3,2


In [9]:
datim = parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)


In [10]:
datim[('Female','<1')] = 0
datim[('Female','1-4')] = 0
datim[('Female','5-9')] = 0
datim[('Female','10-14')] = 0
datim[('Female','15-17')] = 0
datim[('Male','<1')] = 0
datim[('Male','1-4')] = 0
datim[('Male','5-9')] = 0
datim[('Male','10-14')] = 0
datim[('Male','15-17')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim = datim.reindex(columns= columns_orientation)
datim

Gender                       Female                         Male          \
age_ovc                          <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                    
delmas                            0   0   0     0     0   1    0   0   0   
desdunes                          0   0   0     0     0  12    0   0   0   
dessalines                        0   0   0     0     0   3    0   0   0   
grande-saline                     0   0   0     0     0   6    0   0   0   
gressier                          0   0   0     0     0   8    0   0   0   
kenscoff                          0   0   0     0     0  24    0   0   0   
la_chapelle                       0   0   0     0     0   1    0   0   0   
montrouis                         0   0   0     0     0   3    0   0   0   
petion_ville                      0   0   0     0     0   6    0   0   0   
petite_rivire_de_lartibonite      0   0   0     0     0   3    0   0   0   
tabarre                           0   0   0     0     0   7    0   0   0   
verrettes                         0   0   0     0     0   3    0   0   0   

Gender                                        
age_ovc                      10-14 15-17 18+  
commune                                       
delmas                           0     0   0  
desdunes                         0     0   8  
dessalines                       0     0   0  
grande-saline                    0     0   0  
gressier                         0     0   0  
kenscoff                         0     0  10  
la_chapelle                      0     0   0  
montrouis                        0     0   2  
petion_ville                     0     0   1  
petite_rivire_de_lartibonite     0     0   1  
tabarre                          0     0   3  
verrettes                        0     0   0

# REPORT

In [11]:
datim.to_excel(f"./caregiver_results/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)
found_FAP.to_excel(f"./caregiver_results/datim_caregiver_{date_du_jour}.xlsx",na_rep="",index=False)
#datim.to_excel(f"./caregiver_results/datim_parenting_caregiver.xlsx",na_rep="",index=True)

In [12]:
missing_FAP.to_excel(f"./missing/missing_FAP_{date_du_jour}.xlsx", na_rep="",index=False)